**Eric Meinhardt / emeinhardt@ucsd.edu**

In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os

# Overview

This notebook documents the process of creating an n-gram model from the Fisher corpus using `kenlm`.

# Basic structure of the data / code for interfacing with it

In [3]:
repo_dir = '/mnt/cube/home/AD/emeinhar/fisher-lm'

In [4]:
fisher_dir = "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran"

In [5]:
os.getcwd()

'/mnt/cube/home/AD/emeinhar/fisher-lm'

In [6]:
os.chdir(fisher_dir)

In [7]:
%ls

data/  doc/  index.html*


In [8]:
os.chdir(os.path.join(fisher_dir, 'data'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data'

In [9]:
%ls

bbn_orig/  trans/


Here we can see that there are two representations of the Fisher Corpus Part 2.

## Main set of transcriptions

In [10]:
os.chdir(os.path.join(fisher_dir, 'data' + '/' + 'trans'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans'

In [11]:
main_data_dir = '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans'

In [12]:
%ls

058/  063/  068/  073/  078/  083/  088/  093/  098/  103/  108/  113/
059/  064/  069/  074/  079/  084/  089/  094/  099/  104/  109/  114/
060/  065/  070/  075/  080/  085/  090/  095/  100/  105/  110/  115/
061/  066/  071/  076/  081/  086/  091/  096/  101/  106/  111/  116/
062/  067/  072/  077/  082/  087/  092/  097/  102/  107/  112/


In [13]:
trans_folders = os.listdir()

In [14]:
os.chdir(os.path.join(main_data_dir, '058'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058'

In [15]:
%ls

fe_03_05851.txt*  fe_03_05864.txt*  fe_03_05877.txt*  fe_03_05890.txt*
fe_03_05852.txt*  fe_03_05865.txt*  fe_03_05878.txt*  fe_03_05891.txt*
fe_03_05853.txt*  fe_03_05866.txt*  fe_03_05879.txt*  fe_03_05892.txt*
fe_03_05854.txt*  fe_03_05867.txt*  fe_03_05880.txt*  fe_03_05893.txt*
fe_03_05855.txt*  fe_03_05868.txt*  fe_03_05881.txt*  fe_03_05894.txt*
fe_03_05856.txt*  fe_03_05869.txt*  fe_03_05882.txt*  fe_03_05895.txt*
fe_03_05857.txt*  fe_03_05870.txt*  fe_03_05883.txt*  fe_03_05896.txt*
fe_03_05858.txt*  fe_03_05871.txt*  fe_03_05884.txt*  fe_03_05897.txt*
fe_03_05859.txt*  fe_03_05872.txt*  fe_03_05885.txt*  fe_03_05898.txt*
fe_03_05860.txt*  fe_03_05873.txt*  fe_03_05886.txt*  fe_03_05899.txt*
fe_03_05861.txt*  fe_03_05874.txt*  fe_03_05887.txt*
fe_03_05862.txt*  fe_03_05875.txt*  fe_03_05888.txt*
fe_03_05863.txt*  fe_03_05876.txt*  fe_03_05889.txt*


In [16]:
os.listdir()[0]
os.listdir()[0][6:11]

'fe_03_05851.txt'

'05851'

In [17]:
%cat -n fe_03_05851.txt | head -60

     1	# fe_03_05851.sph
     2	# Transcribed at the LDC
     3	
     4	56.38 61.03 A: i mean no money is very important definitely and a million dollars is a dream come true for me i mean 
     5	
     6	61.19 64.71 A: i can actually imagine the kind of things i can buy 
     7	
     8	64.41 65.38 B: yeah 
     9	
    10	65.57 69.14 A: travel and do whatever i want to in life but then again 
    11	
    12	69.61 70.13 A: i just 
    13	
    14	70.25 73.34 A: can't i mean stop talking to my best friend that's like 
    15	
    16	73.54 76.25 A: i can't do that i mean i've been so close to her 
    17	
    18	76.52 78.31 A: she has been my friend since my school days 
    19	
    20	79.01 81.04 A: so i share almost everything with her 
    21	
    22	81.15 81.91 B: mhm th- 
    23	
    24	81.36 85.99 A: whatever problems whatever successes that i've faced in life i mean everything i have to share with her 
    25	
    26	86.36 89.54 B: and they know so much about you and 
    27	
    28

Per the documentation, each five digit number before the file extension is a unique call id.

### Organizing call transcription files, a unique call ID, and call transcription file location

**Let's create a relation between filename, call id, and location.**

In [18]:
os.path.join(main_data_dir, '058' + '/' + 'fe_03_05851.txt')

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'

In [19]:
def filenameToRelation_orig(fn, curr_dir):
    return {'filename':fn,
            'call_id':fn[6:11],
            'location':os.path.join(main_data_dir, curr_dir + '/' + fn)}

In [46]:
os.chdir(main_data_dir)
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans'

In [47]:
main_call_relation = []
for folder in trans_folders:
    os.chdir(folder)
    for file in os.listdir():
        main_call_relation.append(filenameToRelation_orig(file, folder))
    os.chdir('..')

In [48]:
main_call_relation[:3]

[{'filename': 'fe_03_05851.txt',
  'call_id': '05851',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'},
 {'filename': 'fe_03_05852.txt',
  'call_id': '05852',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05852.txt'},
 {'filename': 'fe_03_05853.txt',
  'call_id': '05853',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05853.txt'}]

### Organizing transcriptions of each call

Each line of a transcribed call file is either
 - a comment (prefixed with `#`), only ever appearing in the top two lines of the file
 - blank
 - or a transcription line.

A transcription line consists of
 - a two decimal point float indicating the time of the start of the transcribed speech in the line
 - a two decimal point float indicating the time of the end of the transcribed speech in the line
 - two characters indicating which of two speakers produced the transcribed speech in the line
 - a sequence of transcribed speech wordforms (all lowercase)

Each of these elements is separated from the other by a space.

Transcribed wordform sequences that transcribers were unsure of what exactly was said are surrounded by double parentheses, with the transcriber's best guess (if any) inside the parentheses e.g. 
  - `295.11 298.23 A: (( no way )) money can be substituted for her no way`
  - `335.99 341.73 B: (( )) on friday and i have to work it's my last day with them i teach preschool so`

In [49]:
%cat -n ./058/fe_03_05851.txt | head -20

     1	# fe_03_05851.sph
     2	# Transcribed at the LDC
     3	
     4	56.38 61.03 A: i mean no money is very important definitely and a million dollars is a dream come true for me i mean 
     5	
     6	61.19 64.71 A: i can actually imagine the kind of things i can buy 
     7	
     8	64.41 65.38 B: yeah 
     9	
    10	65.57 69.14 A: travel and do whatever i want to in life but then again 
    11	
    12	69.61 70.13 A: i just 
    13	
    14	70.25 73.34 A: can't i mean stop talking to my best friend that's like 
    15	
    16	73.54 76.25 A: i can't do that i mean i've been so close to her 
    17	
    18	76.52 78.31 A: she has been my friend since my school days 
    19	
    20	79.01 81.04 A: so i share almost everything with her 


In [50]:
def get_raw_transcription_lines(callfile_fn):
    lines = []
    with open(callfile_fn, 'r', encoding='latin-1') as the_file:
        for line in the_file:
            if line[0] != '#' and line.rstrip() != '':
                lines.append(line.rstrip())
    return lines

In [51]:
ls = get_raw_transcription_lines('./058/fe_03_05851.txt')
ls[:5]

['56.38 61.03 A: i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
 '61.19 64.71 A: i can actually imagine the kind of things i can buy',
 '64.41 65.38 B: yeah',
 '65.57 69.14 A: travel and do whatever i want to in life but then again',
 '69.61 70.13 A: i just']

In [52]:
round(4.649999999999999, 2)

4.65

In [53]:
def remove_double_parens(speech):
    speech_wordform_seq = speech.split(' ')
    filtered = [w for w in speech_wordform_seq if w != '((' and w != '))']
    return ' '.join(filtered)

In [54]:
remove_double_parens('(( no way )) money can be substituted for her no way')
remove_double_parens("(( )) on friday and i have to work it's my last day with them i teach preschool so")

'no way money can be substituted for her no way'

"on friday and i have to work it's my last day with them i teach preschool so"

In [55]:
def parse_transcription_line(line):
    parts = line.split(' ')
    start = float(parts[0])
    end = float(parts[1])
    duration = round(end - start, 2)
    speaker = parts[2][0]
    speech_raw = ' '.join(parts[3:])
    speech_no_parens = remove_double_parens(speech_raw)
    line_relation = {'start':start,
                     'end':end,
                     'duration':duration,
                     'speaker':speaker,
                     'speech_raw':speech_raw,
                     'speech':speech_no_parens}
    return line_relation

In [56]:
parse_transcription_line(ls[0])

{'start': 56.38,
 'end': 61.03,
 'duration': 4.65,
 'speaker': 'A',
 'speech_raw': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
 'speech': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean'}

In [57]:
def get_transcriptions_main(call_id):
    call_rel_match = [c for c in main_call_relation if c['call_id'] == call_id]
    assert len(call_rel_match) == 1, 'No matching call with call_id = {0}'.format(call_id)
    call_rel = call_rel_match[0]
#     call_fn = call_rel['filename']
    parsed_lines = [parse_transcription_line(l) 
                    for l in get_raw_transcription_lines(call_rel['location'])]
    return parsed_lines

In [58]:
get_transcriptions_main('05851')[:3]

[{'start': 56.38,
  'end': 61.03,
  'duration': 4.65,
  'speaker': 'A',
  'speech_raw': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
  'speech': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean'},
 {'start': 61.19,
  'end': 64.71,
  'duration': 3.52,
  'speaker': 'A',
  'speech_raw': 'i can actually imagine the kind of things i can buy',
  'speech': 'i can actually imagine the kind of things i can buy'},
 {'start': 64.41,
  'end': 65.38,
  'duration': 0.97,
  'speaker': 'B',
  'speech_raw': 'yeah',
  'speech': 'yeah'}]

### Some summary / sanity-checking statistics

How many calls are in the main transcripts?

In [59]:
call_ids = [c['call_id'] for c in main_call_relation]
len(call_ids)

5849

How many word tokens are in the main transcripts?

In [60]:
def tokenize_speech_in_line(line_rel):
    return line_rel['speech'].split(' ')

def count_word_tokens_in_line(line_rel):
    return len(tokenize_speech_in_line(line_rel))

def count_word_tokens_in_call(call_id):
    return sum([count_word_tokens_in_line(l) for l in get_transcriptions_main(call_id)])

In [61]:
#9.9s on wittgenstein
sum([count_word_tokens_in_call(call_id) for call_id in call_ids])

11035101

How many word types are in the main transcripts?

In [62]:
from functools import reduce

In [63]:
union = lambda sets: reduce(set.union, sets)

union([{0,1},{1,2},{3}])

{0, 1, 2, 3}

In [64]:
def word_types_in_line(line_rel):
    return set(tokenize_speech_in_line(line_rel))

def word_types_in_call(call_id):
    return union([word_types_in_line(l) for l in get_transcriptions_main(call_id)])

In [65]:
#20.0s on wittgenstein
vocabulary_main = union([word_types_in_call(c) for c in call_ids])
len(vocabulary_main)

48797

In [66]:
list(vocabulary_main)[:100]

['',
 'account',
 'ardy',
 'capacity',
 'surveillance',
 'merchantville',
 'differing',
 'jewel',
 'deploy',
 'schlocky',
 'blather',
 'extraordinary',
 'allergy',
 "he'd",
 'portrait',
 'serbo',
 'sarajevo',
 'collar',
 'rag',
 'seeping',
 'feasibility',
 'kiki',
 'genera-',
 'during',
 'mainly',
 'dirges',
 'georgeanne',
 'tab',
 'redux',
 'comtrex',
 'public-',
 'escapologist',
 'retrievers',
 'limits',
 'liveable',
 'contempt',
 'dressmaker',
 'c._d._s.',
 'refunded',
 'logically',
 'matures',
 'heats',
 'visions',
 'exiled',
 'colan',
 'quan-',
 'wakeman',
 'marketplace',
 'chintzy',
 'skeleton',
 'calter',
 'wherev-',
 'r._e._i.',
 'inventories',
 "region's",
 'chris-',
 'trickling',
 'weedner',
 'streaks',
 'gauging',
 'cleverly',
 'it',
 'surround',
 'forbidden',
 'supplier',
 'davy',
 'jinks',
 'kudlow',
 'informant',
 "scott's",
 'forks',
 'shivering',
 'butterworth',
 'profit',
 'all?',
 'amazes',
 'trickster',
 'reign',
 'expl-',
 'dong',
 'existentialism',
 "teriyaki's",
 

In [67]:
%%capture
#cell magic above suppresses useless output of file.write

vocabulary_main_fn = 'fisher_vocabulary_main.txt'

vocabulary_main_fp = os.path.join(repo_dir, vocabulary_main_fn)

#5.29s on wittgenstein
with open(vocabulary_main_fp, 'w') as file:
    for w in vocabulary_main:
        file.write("{0}\n".format(w));

In [68]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_vocabulary_main.txt | head -20

     1	
     2	account
     3	ardy
     4	capacity
     5	surveillance
     6	merchantville
     7	differing
     8	jewel
     9	deploy
    10	schlocky
    11	blather
    12	extraordinary
    13	allergy
    14	he'd
    15	portrait
    16	serbo
    17	sarajevo
    18	collar
    19	rag
    20	seeping
cat: write error: Broken pipe


In [69]:
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans'

### Naive utterance sequence

In [70]:
def get_utterances_from_call(call_id):
    return [l['speech'] for l in get_transcriptions_main(call_id)]

In [71]:
call_ids[0]

'05851'

In [72]:
get_utterances_from_call('05851')[:10]

['i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
 'i can actually imagine the kind of things i can buy',
 'yeah',
 'travel and do whatever i want to in life but then again',
 'i just',
 "can't i mean stop talking to my best friend that's like",
 "i can't do that i mean i've been so close to her",
 'she has been my friend since my school days',
 'so i share almost everything with her',
 'mhm th-']

In [73]:
#9.2s on wittgenstein
utterances_main = []
for c in call_ids:
    utterances_main.extend( get_utterances_from_call(c) )
len(utterances_main)

1113586

In [74]:
utterances_main[0]

'i mean no money is very important definitely and a million dollars is a dream come true for me i mean'

In [75]:
%pwd

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans'

In [76]:
repo_dir

'/mnt/cube/home/AD/emeinhar/fisher-lm'

In [77]:
%%capture
#cell magic above suppresses useless output of file.write

fisher_utterances_main_fn = 'fisher_utterances_main.txt'

fisher_utterances_main_path = os.path.join(repo_dir, fisher_utterances_main_fn)

#2m 2s on wittgenstein
with open(fisher_utterances_main_path, 'w') as file:
    for utterance in utterances_main:
        file.write("{0}\n".format(utterance));

In [78]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt | head -20

     1	i mean no money is very important definitely and a million dollars is a dream come true for me i mean
     2	i can actually imagine the kind of things i can buy
     3	yeah
     4	travel and do whatever i want to in life but then again
     5	i just
     6	can't i mean stop talking to my best friend that's like
     7	i can't do that i mean i've been so close to her
     8	she has been my friend since my school days
     9	so i share almost everything with her
    10	mhm th-
    11	whatever problems whatever successes that i've faced in life i mean everything i have to share with her
    12	and they know so much about you and
    13	yeah i know
    14	everything i mean any little problem any little
    15	thing that you have in your life you can always go and talk to her about it
    16	mhm
    17	really nice it really feels nice you know you can share your burdens with her your joys with her your sorrows with her
    18	seriously money is definitely important but th- but then
 

### All transcriptions

In [79]:
main_call_relation[:3]

[{'filename': 'fe_03_05851.txt',
  'call_id': '05851',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'},
 {'filename': 'fe_03_05852.txt',
  'call_id': '05852',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05852.txt'},
 {'filename': 'fe_03_05853.txt',
  'call_id': '05853',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05853.txt'}]

In [80]:
call_ids[0]

'05851'

In [81]:
get_transcriptions_main('05851')[:3]

[{'start': 56.38,
  'end': 61.03,
  'duration': 4.65,
  'speaker': 'A',
  'speech_raw': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
  'speech': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean'},
 {'start': 61.19,
  'end': 64.71,
  'duration': 3.52,
  'speaker': 'A',
  'speech_raw': 'i can actually imagine the kind of things i can buy',
  'speech': 'i can actually imagine the kind of things i can buy'},
 {'start': 64.41,
  'end': 65.38,
  'duration': 0.97,
  'speaker': 'B',
  'speech_raw': 'yeah',
  'speech': 'yeah'}]

In [82]:
def dictionary_product(dictA, dictB):
    '''
    Given two dictionaries dictA and dictB, this non-statefully 
    creates a 'merged' dictionary.
    
    Note that dictA and dictB must either have disjoint keys,
    or every key they share must map to the same value.
    '''
    keys_A = set(dictA.keys())
    keys_B = set(dictB.keys())
    common_keys = set(keys_A) & set(keys_B)
    conflicting_common_keys = {k for k in common_keys if dictA[k] != dictB[k]}
    assert conflicting_common_keys == set(), 'The dictionaries must have either zero common keys, or all common keys must map to the same value. Conflicting common keys: {0}'.format(conflicting_common_keys)
#     assert common_keys == set(), 'The dictionaries must have no common keys. They share {0}'.format(common_keys)
    
    dictC = dict()
    dictC.update({k:dictA[k] for k in keys_A})
    dictC.update({k:dictB[k] for k in keys_B})
    return dictC

In [83]:
main_call_relation[0]

{'filename': 'fe_03_05851.txt',
 'call_id': '05851',
 'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'}

In [84]:
main_transcription_relation = []

for call_relation in main_call_relation:
    for transcription_relation in get_transcriptions_main(call_relation['call_id']):
        new_transcription_rel = dictionary_product(transcription_relation, call_relation)
        main_transcription_relation.append(new_transcription_rel)

In [85]:
len(main_transcription_relation)

1113586

In [86]:
main_transcription_relation[:5]

[{'speaker': 'A',
  'speech': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
  'start': 56.38,
  'end': 61.03,
  'duration': 4.65,
  'speech_raw': 'i mean no money is very important definitely and a million dollars is a dream come true for me i mean',
  'call_id': '05851',
  'filename': 'fe_03_05851.txt',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'},
 {'speaker': 'A',
  'speech': 'i can actually imagine the kind of things i can buy',
  'start': 61.19,
  'end': 64.71,
  'duration': 3.52,
  'speech_raw': 'i can actually imagine the kind of things i can buy',
  'call_id': '05851',
  'filename': 'fe_03_05851.txt',
  'location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt'},
 {'speaker': 'B',
  'speech': 'yeah',
  'start': 64.41,
  'end': 65.38,
  'duration': 0.97,
  'speech_raw': 'yeah',
  'call_id': '05851',
  'filenam

In [87]:
import json

fisher_utterances_relation_main_fn = 'fisher_utterances_relation_main.json'

fisher_utterances_relation_main_path = os.path.join(repo_dir, fisher_utterances_relation_main_fn)

#22.2s on wittgenstein
with open(fisher_utterances_relation_main_path, 'w', encoding='utf8') as json_file:
    json.dump(main_transcription_relation, json_file, indent=1, ensure_ascii=False)

In [88]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_relation_main.json | head -20

     1	[
     2	 {
     3	  "speaker": "A",
     4	  "speech": "i mean no money is very important definitely and a million dollars is a dream come true for me i mean",
     5	  "start": 56.38,
     6	  "end": 61.03,
     7	  "duration": 4.65,
     8	  "speech_raw": "i mean no money is very important definitely and a million dollars is a dream come true for me i mean",
     9	  "call_id": "05851",
    10	  "filename": "fe_03_05851.txt",
    11	  "location": "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/trans/058/fe_03_05851.txt"
    12	 },
    13	 {
    14	  "speaker": "A",
    15	  "speech": "i can actually imagine the kind of things i can buy",
    16	  "start": 61.19,
    17	  "end": 64.71,
    18	  "duration": 3.52,
    19	  "speech_raw": "i can actually imagine the kind of things i can buy",
    20	  "call_id": "05851",
cat: write error: Broken pipe


## The `bbn` transcriptions

For context, read
 - the last page of Cieri et al. 2004 ("The Fisher Corpus: a Resource for the Next Generations of Speech-to-Text", https://www.ldc.upenn.edu/sites/www.ldc.upenn.edu/files/lrec2004-fisher-corpus.pdf)
 - https://catalog.ldc.upenn.edu/docs/LDC2005T19/bbn_trans_readme.txt

In [89]:
os.chdir(os.path.join(fisher_dir, 'data'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data'

In [90]:
%ls

bbn_orig/  trans/


In [91]:
%cd bbn_orig/

/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig


In [92]:
bbn_data_dir = '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig'

In [93]:
%ls

058/  063/  068/  073/  078/  083/  088/  093/  098/  103/  108/  113/
059/  064/  069/  074/  079/  084/  089/  094/  099/  104/  109/
060/  065/  070/  075/  080/  085/  090/  095/  100/  105/  110/
061/  066/  071/  076/  081/  086/  091/  096/  101/  106/  111/
062/  067/  072/  077/  082/  087/  092/  097/  102/  107/  112/


In [94]:
bbn_folders = os.listdir()

In [95]:
os.chdir(os.path.join(bbn_data_dir, '058'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

In [96]:
%ls

auto-segmented/  originals/  rejected/


Note that, per `https://catalog.ldc.upenn.edu/docs/LDC2005T19/bbn_trans_readme.txt`, 
 - each folder contains a group of (up to?) 100 calls with a shared 3 digit prefix in their call id.
 - subsets 114, 115, and 116 are not here, because those 300 transcripts were done by the LDC.
 
Within each folder, `/originals/` contains `.txo` transcriptions produced by annotators at WordWave; these files were the input to an automatic segmentation process that created the `.ana` and `.trn` files in `/auto-segmented/` folders. `/rejected/` folders contain `.ana` and `.trn` data for utterances that the automatic segmentation process could not handle.

That means the transcription data of interest for this project are in `.trn` files in both `/auto-segmented/` and `/rejected/` folders and will need to be combined.

In [97]:
%cd auto-segmented/

/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented


In [98]:
%ls

fe_03_05863.ana*  fe_03_05876.trn*  fe_03_05888.ana*  fe_03_05889.trn*
fe_03_05863.trn*  fe_03_05887.ana*  fe_03_05888.trn*  fe_03_05898.ana*
fe_03_05876.ana*  fe_03_05887.trn*  fe_03_05889.ana*  fe_03_05898.trn*


In [99]:
%cat -n fe_03_05876.trn | head -20

     1	H- HI THIS IS DEBBIE  (fe_03_05876-A-0001)
     2	HI SYLVIA  (fe_03_05876-A-0002)
     3	[LAUGHTER] I'M JUST TRYING TO GET OVER THIS QUESTION  (fe_03_05876-A-0003)
     4	UH NO IT'S MY SECOND CALL BUT THEY JUST SEEM TO GET MO- [LAUGHTER] MORE AND MORE RIDICULOUS  (fe_03_05876-A-0005)
     5	OH REALLY [LAUGHTER] UM  (fe_03_05876-A-0006)
     6	WELL NO MY BEST FRIEND UM  (fe_03_05876-A-0007)
     7	I DON'T KNOW MY INSTINCT MY INITIAL INSTINCT IS TO SAY NO UH YOU KNOW  (fe_03_05876-A-0008)
     8	HOWEVER I GUESS [LAUGHTER] IF I HAD TO THINK ABOUT IT A WHILE  (fe_03_05876-A-0009)
     9	SEE I COULD DO SO MUCH WITH A MILLION DOLLARS HELP SO MANY PEOPLE BUT [LIPSMACK]  (fe_03_05876-A-0010)
    10	I DON'T KNOW IT JUST DOESN'T IT DOESN'T FEEL RIGHT TO TO TAKE TO HAVE THIS STRING ATTACHED YOU KNOW FIRST OF ALL  (fe_03_05876-A-0011)
    11	[LAUGHTER] SO DO I. THAT'S RIGHT  (fe_03_05876-A-0012)
    12	YOU'RE RIGHT  (fe_03_05876-A-0013)
    13	THAT'S TRUE [NOISE]  (fe_03_05876-A-0014)
    1

### Organizing call transcription files, a unique call ID, and call transcription file location

**Let's create a relation between call id, associated filenames, and locations.**

In [100]:
%cd ..

/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058


In [101]:
%pwd

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

In [102]:
%ls

auto-segmented/  originals/  rejected/


In [103]:
set(os.listdir('./auto-segmented/')) == set(os.listdir('./rejected/'))

True

In [104]:
os.listdir('./auto-segmented/')

['fe_03_05863.ana',
 'fe_03_05863.trn',
 'fe_03_05876.ana',
 'fe_03_05876.trn',
 'fe_03_05887.ana',
 'fe_03_05887.trn',
 'fe_03_05888.ana',
 'fe_03_05888.trn',
 'fe_03_05889.ana',
 'fe_03_05889.trn',
 'fe_03_05898.ana',
 'fe_03_05898.trn']

In [105]:
'fe_03_05898.trn'[6:11]

'05898'

In [106]:
def fn_to_call_id_bbn(bbn_fn):
    return bbn_fn[6:11]

In [107]:
def get_call_ids_in_group_dir(group_dir):
    autosegmented_files = set(os.listdir('./auto-segmented/'))
    rejected_files = set(os.listdir('./rejected'))
    autosegmented_call_ids = set(map(fn_to_call_id_bbn,
                                     autosegmented_files))
    rejected_call_ids = set(map(fn_to_call_id_bbn,
                                rejected_files))
    return sorted(list(autosegmented_call_ids | rejected_call_ids))

In [108]:
get_call_ids_in_group_dir('.')

['05863', '05876', '05887', '05888', '05889', '05898']

In [109]:
def call_id_to_relation_bbn(call_id, group_dir):
    fn_prefix = 'fe_03_'
    suffix_a = '.ana'
    suffix_b = '.trn'
    ana_fn = fn_prefix + call_id + suffix_a
    trn_fn = fn_prefix + call_id + suffix_b
    loc_prefix = os.path.join(bbn_data_dir, group_dir)
    return {'call_id':call_id,
            'trn_fn':trn_fn,
            'ana_fn':ana_fn,
            'auto-segmented_trn_location':os.path.join(loc_prefix, 'auto-segmented/' + trn_fn),
            'rejected_trn_location':os.path.join(loc_prefix, 'rejected/' + trn_fn) if os.path.isfile(os.path.join(loc_prefix, 'rejected/' + trn_fn)) else '',
            'auto-segmented_ana_location':os.path.join(loc_prefix, 'auto-segmented/' + ana_fn),
            'rejected_ana_location':os.path.join(loc_prefix, 'rejected/' + ana_fn) if os.path.isfile(os.path.join(loc_prefix, 'rejected/' + ana_fn)) else '',}

In [110]:
%pwd

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

In [111]:
os.listdir('./auto-segmented/')
os.listdir('./rejected/')

['fe_03_05863.ana',
 'fe_03_05863.trn',
 'fe_03_05876.ana',
 'fe_03_05876.trn',
 'fe_03_05887.ana',
 'fe_03_05887.trn',
 'fe_03_05888.ana',
 'fe_03_05888.trn',
 'fe_03_05889.ana',
 'fe_03_05889.trn',
 'fe_03_05898.ana',
 'fe_03_05898.trn']

['fe_03_05863.ana',
 'fe_03_05863.trn',
 'fe_03_05876.ana',
 'fe_03_05876.trn',
 'fe_03_05887.ana',
 'fe_03_05887.trn',
 'fe_03_05888.ana',
 'fe_03_05888.trn',
 'fe_03_05889.ana',
 'fe_03_05889.trn',
 'fe_03_05898.ana',
 'fe_03_05898.trn']

In [112]:
call_id_to_relation_bbn('05863', '058')

loc = call_id_to_relation_bbn('05863', '058')['auto-segmented_trn_location']
loc

{'call_id': '05863',
 'trn_fn': 'fe_03_05863.trn',
 'ana_fn': 'fe_03_05863.ana',
 'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
 'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
 'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
 'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'}

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn'

In [113]:
os.chdir(bbn_data_dir)
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig'

In [114]:
bbn_call_relation = []
for folder in bbn_folders:
    os.chdir(folder)
    for call_id in get_call_ids_in_group_dir('.'):
        bbn_call_relation.append(call_id_to_relation_bbn(call_id, folder))
    os.chdir('..')

In [115]:
bbn_call_relation[:3]

[{'call_id': '05863',
  'trn_fn': 'fe_03_05863.trn',
  'ana_fn': 'fe_03_05863.ana',
  'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
  'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
  'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
  'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'},
 {'call_id': '05876',
  'trn_fn': 'fe_03_05876.trn',
  'ana_fn': 'fe_03_05876.ana',
  'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05876.trn',
  'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_

### Organizing transcriptions of each call

In [116]:
os.chdir(os.path.join(bbn_data_dir, '058'))
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

In [117]:
%ls auto-segmented/

fe_03_05863.ana*  fe_03_05876.trn*  fe_03_05888.ana*  fe_03_05889.trn*
fe_03_05863.trn*  fe_03_05887.ana*  fe_03_05888.trn*  fe_03_05898.ana*
fe_03_05876.ana*  fe_03_05887.trn*  fe_03_05889.ana*  fe_03_05898.trn*


#### Parsing transcription lines

In [118]:
%cat -n ./auto-segmented/fe_03_05863.trn | head -25

     1	HI [LAUGHTER]  (fe_03_05863-A-0001)
     2	HI MY NAME IS ZELDA  (fe_03_05863-A-0002)
     3	HI MONIQUE  (fe_03_05863-A-0003)
     4	I'M FINE AND YOU  (fe_03_05863-A-0004)
     5	GOOD  (fe_03_05863-A-0005)
     6	OKAY UM  (fe_03_05863-A-0006)
     7	I WOULD NOT DO IT  (fe_03_05863-A-0007)
     8	[NOISE] RIGHT I WOULDN'T UM DO YOU KID- ESP- UH TODAY I MEAN  (fe_03_05863-A-0008)
     9	PEOPLE LOOK AT MONEY MONEY IS EVERYTHING IT'S LIKE THEY THINK THAT MONEY'S GOING TO RESOLVE ALL THEIR PROBLEMS AND A MILLION DOLLARS TODAY IS NOT A LOT OF MONEY AND I WOULD AND THAT'S A BETRAYAL TO ME [NOISE]  (fe_03_05863-A-0009)
    10	RIGHT RIGHT  (fe_03_05863-A-0010)
    11	NO  (fe_03_05863-A-0011)
    12	I THINK HUMAN BEINGS ARE PR- UM [LIPSMACK] FRIENDSHIP RELAT- RELATIONSHIPS WITH PEOPLE ARE A LOT MORE IMPORTANT THAN MONEY  (fe_03_05863-A-0012)
    13	THEN YOU WEREN'T REALLY THEIR BEST FRIEND EITHER  (fe_03_05863-A-0013)
    14	RIGHT RIGHT  (fe_03_05863-A-0014)
    15	NO  (fe_03_05863-A-0015)


In [119]:
%cat -n ./rejected/fe_03_05863.trn | head -20

     1	BUT UM (fe_03_05863-A-0021)
     2	NO NO (fe_03_05863-A-0045)
     3	UM (fe_03_05863-A-0065)
     4	[LIPSMACK] OH (fe_03_05863-A-0066)
     5	REALLY (fe_03_05863-A-0105)
     6	YEAH MHM (fe_03_05863-A-0109)
     7	THAT'S WHAT I SAY MHM (fe_03_05863-A-0113)
     8	YOU KNOW (fe_03_05863-A-0121)
     9	YEAH (fe_03_05863-A-0122)
    10	ON ME THERE OOH THAT'S WHY UM I'M I'M LOSING YA (fe_03_05863-A-0125)
    11	[NOISE] HELLO HOW ARE YOU DOING MY (fe_03_05863-B-0002)
    12	HOW'RE YOU DOING (fe_03_05863-B-0004)
    13	MM MM (fe_03_05863-B-0019)
    14	[LAUGHTER] OH BOY [SIGH] (fe_03_05863-B-0021)
    15	LET'S SEE LET'S SEE (fe_03_05863-B-0022)
    16	YOU KNOW (fe_03_05863-B-0027)
    17	[NOISE] (fe_03_05863-B-0029)
    18	UH-HUH (fe_03_05863-B-0034)
    19	YOU LIVE IN CALIFORNIA (fe_03_05863-B-0039)
    20	MAN (fe_03_05863-B-0045)


As indicated in `https://catalog.ldc.upenn.edu/docs/LDC2005T19/bbn_trans_readme.txt`, `.trn` files are in .SNOR format:
 - mono-case with no syntactic punctuation
 - each line is one "utterance"
 - each line ends with a unique utterance ID in parentheses

Non-speech noises seem to be indicated by square brackets (e.g. `[LAUGHTER]`).

In [120]:
def parse_trn_line_bbn(bbn_line):
    parts = bbn_line.split(' ')
    
#     utterance_id = parts[-1]
    utterance_id = parts[-1][1:-1] #strip parentheses off
    
    speech_raw_words = parts[:-1]
    speech_raw = ' '.join(speech_raw_words)
    speech_raw = speech_raw.rstrip()
    
    if utterance_id == "(fe_03_07397-A-0028":
        utterance_id = "fe_03_07397-A-0028"
        speech_raw_words += ['S']
        speech_raw += ' S'
    elif utterance_id == "(fe_03_10453-A-0023":
        utterance_id = "fe_03_10453-A-0023"
        speech_raw_words += ['S']
        speech_raw += ' S'
    
    speech_filtered_words = [w for w in speech_raw_words if len(w) > 0 and w[0] != '[' and w[-1] != ']']
    speech = ' '.join(speech_filtered_words)
    speech = speech.rstrip()
        
    line_rel = {'speech_raw':speech_raw,
                'speech':speech,
                'utterance_id':utterance_id}
    
    return line_rel

parse_trn_line_bbn('[LAUGHTER] OH BOY [SIGH] (fe_03_05863-B-0021)')
parse_trn_line_bbn("[LAUGHTER] THE DEAD S(fe_03_10453-A-0023)")

{'speech_raw': '[LAUGHTER] OH BOY [SIGH]',
 'speech': 'OH BOY',
 'utterance_id': 'fe_03_05863-B-0021'}

{'speech_raw': '[LAUGHTER] THE DEAD S',
 'speech': 'THE DEAD S',
 'utterance_id': 'fe_03_10453-A-0023'}

As indicated in `https://catalog.ldc.upenn.edu/docs/LDC2005T19/bbn_trans_readme.txt`, each line of an `.ana` file indicates the start and end times (in samples) of each utterance:
 - the last column is an utterance id
 - the second or third from last column indicates the sample range of the utterance

In [121]:
%cat -n ./rejected/fe_03_05863.ana | head -20

     1	fe_03_05863 -c 1 -t NIST_1A -f 966480-974640 -o fe_03_05863-A-0021
     2	fe_03_05863 -c 1 -t NIST_1A -f 1821360-1826880 -o fe_03_05863-A-0045
     3	fe_03_05863 -c 1 -t NIST_1A -f 2573120-2579840 -o fe_03_05863-A-0065
     4	fe_03_05863 -c 1 -t NIST_1A -f 2592080-2598160 -o fe_03_05863-A-0066
     5	fe_03_05863 -c 1 -t NIST_1A -f 4225360-4233200 -o fe_03_05863-A-0105
     6	fe_03_05863 -c 1 -t NIST_1A -f 4392240-4404720 -o fe_03_05863-A-0109
     7	fe_03_05863 -c 1 -t NIST_1A -f 4482880-4492720 -o fe_03_05863-A-0113
     8	fe_03_05863 -c 1 -t NIST_1A -f 4687600-4694240 -o fe_03_05863-A-0121
     9	fe_03_05863 -c 1 -t NIST_1A -f 4702240-4710880 -o fe_03_05863-A-0122
    10	fe_03_05863 -c 1 -t NIST_1A -f 4772000-4799200 -o fe_03_05863-A-0125
    11	fe_03_05863 -c 2 -t NIST_1A -f 20240-32480 -o fe_03_05863-B-0002
    12	fe_03_05863 -c 2 -t NIST_1A -f 71360-80240 -o fe_03_05863-B-0004
    13	fe_03_05863 -c 2 -t NIST_1A -f 755440-762000 -o fe_03_05863-B-0019
    14	fe_03_05863 -c 2 

In [122]:
"fe_03_05863 -c 2 -t NIST_1A -f 1829200-1836720 -o fe_03_05863-B-0045".split(' ')
"fe_03_05863 -c 2 -t NIST_1A -f 1829200-1836720 -o fe_03_05863-B-0045".split(' ')[-3]
"fe_03_05863 -c 2 -t NIST_1A -f 1829200-1836720 -o fe_03_05863-B-0045".split(' ')[-3].split('-')

['fe_03_05863',
 '-c',
 '2',
 '-t',
 'NIST_1A',
 '-f',
 '1829200-1836720',
 '-o',
 'fe_03_05863-B-0045']

'1829200-1836720'

['1829200', '1836720']

In [123]:
def parse_ana_line_bbn(bbn_line):
    parts = bbn_line.split(' ')
    
#     utterance_id = parts[-1]
    utterance_id = parts[-1]
    
    audio_ext = '.sph'
    audio_fn = parts[0] + audio_ext
    
    channel = int(parts[2])
    
    ch_type = parts[4]
    
    utterance_sample_range = parts[-3]
    
    start_sample = int(utterance_sample_range.split('-')[0])
    end_sample = int(utterance_sample_range.split('-')[1])
    duration = end_sample - start_sample
    
    line_rel = {'utterance_id':utterance_id,
                'start':start_sample,
                'end':end_sample,
                'duration':duration,
                'audio_fn':audio_fn,
                'channel':channel,
                'type':ch_type}
    
    return line_rel

parse_ana_line_bbn("fe_03_05863 -c 2 -t NIST_1A -f 1829200-1836720 -o fe_03_05863-B-0045")

{'utterance_id': 'fe_03_05863-B-0045',
 'start': 1829200,
 'end': 1836720,
 'duration': 7520,
 'audio_fn': 'fe_03_05863.sph',
 'channel': 2,
 'type': 'NIST_1A'}

#### Parsing files

In [124]:
def parse_trn_file_bbn(bbn_fp):
    bbn_lines = []
    with open(bbn_fp, 'r') as file:
        for line in file:
            bbn_lines.append(line.rstrip())
    parsed_lines = [parse_trn_line_bbn(l) for l in bbn_lines]
    return parsed_lines

In [125]:
%pwd

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

In [126]:
%ls ./auto-segmented

fe_03_05863.ana*  fe_03_05876.trn*  fe_03_05888.ana*  fe_03_05889.trn*
fe_03_05863.trn*  fe_03_05887.ana*  fe_03_05888.trn*  fe_03_05898.ana*
fe_03_05876.ana*  fe_03_05887.trn*  fe_03_05889.ana*  fe_03_05898.trn*


In [127]:
parse_trn_file_bbn('./auto-segmented/fe_03_05876.trn')

[{'speech_raw': 'H- HI THIS IS DEBBIE',
  'speech': 'H- HI THIS IS DEBBIE',
  'utterance_id': 'fe_03_05876-A-0001'},
 {'speech_raw': 'HI SYLVIA',
  'speech': 'HI SYLVIA',
  'utterance_id': 'fe_03_05876-A-0002'},
 {'speech_raw': "[LAUGHTER] I'M JUST TRYING TO GET OVER THIS QUESTION",
  'speech': "I'M JUST TRYING TO GET OVER THIS QUESTION",
  'utterance_id': 'fe_03_05876-A-0003'},
 {'speech_raw': "UH NO IT'S MY SECOND CALL BUT THEY JUST SEEM TO GET MO- [LAUGHTER] MORE AND MORE RIDICULOUS",
  'speech': "UH NO IT'S MY SECOND CALL BUT THEY JUST SEEM TO GET MO- MORE AND MORE RIDICULOUS",
  'utterance_id': 'fe_03_05876-A-0005'},
 {'speech_raw': 'OH REALLY [LAUGHTER] UM',
  'speech': 'OH REALLY UM',
  'utterance_id': 'fe_03_05876-A-0006'},
 {'speech_raw': 'WELL NO MY BEST FRIEND UM',
  'speech': 'WELL NO MY BEST FRIEND UM',
  'utterance_id': 'fe_03_05876-A-0007'},
 {'speech_raw': "I DON'T KNOW MY INSTINCT MY INITIAL INSTINCT IS TO SAY NO UH YOU KNOW",
  'speech': "I DON'T KNOW MY INSTINCT MY I

limit_output extension: Maximum message size of 20000 exceeded with 33900 characters

In [128]:
def parse_ana_file_bbn(bbn_fp):
    bbn_lines = []
    with open(bbn_fp, 'r') as file:
        for line in file:
            bbn_lines.append(line.rstrip())
    parsed_lines = [parse_ana_line_bbn(l) for l in bbn_lines]
    return parsed_lines

In [129]:
parse_ana_file_bbn('./auto-segmented/fe_03_05876.ana')

[{'utterance_id': 'fe_03_05876-A-0001',
  'start': 19360,
  'end': 33280,
  'duration': 13920,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},
 {'utterance_id': 'fe_03_05876-A-0002',
  'start': 39280,
  'end': 50880,
  'duration': 11600,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},
 {'utterance_id': 'fe_03_05876-A-0003',
  'start': 54720,
  'end': 80080,
  'duration': 25360,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},
 {'utterance_id': 'fe_03_05876-A-0005',
  'start': 94160,
  'end': 131360,
  'duration': 37200,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},
 {'utterance_id': 'fe_03_05876-A-0006',
  'start': 140400,
  'end': 161200,
  'duration': 20800,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},
 {'utterance_id': 'fe_03_05876-A-0007',
  'start': 180480,
  'end': 215200,
  'duration': 34720,
  'audio_fn': 'fe_03_05876.sph',
  'channel': 1,
  'type': 'NIST_1A'},

limit_output extension: Maximum message size of 20000 exceeded with 27931 characters

#### Creating merged representations of each call

In [130]:
#create merged representation of conversation from auto-segmented and rejected .trn and .ana files, sort utterances by utterance ID

In [131]:
bbn_call_relation[0]

{'call_id': '05863',
 'trn_fn': 'fe_03_05863.trn',
 'ana_fn': 'fe_03_05863.ana',
 'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
 'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
 'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
 'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'}

In [132]:
def get_transcriptions_bbn(call_id):
    call_relation_matches = [c for c in bbn_call_relation if c['call_id'] == call_id]
    assert len(call_relation_matches) == 1, 'call id {0} does not have exactly 1 match, but rather {1}'.format(call_id, call_relation_matches)
    call_rel = call_relation_matches[0]
    
    as_trn = parse_trn_file_bbn(call_rel['auto-segmented_trn_location'])
    r_trn = parse_trn_file_bbn(call_rel['rejected_trn_location'])
    merged_trn = sorted(as_trn + r_trn, key=lambda t_rel: t_rel['utterance_id'])
    
    as_ana = parse_ana_file_bbn(call_rel['auto-segmented_ana_location'])
    r_ana = parse_ana_file_bbn(call_rel['rejected_ana_location'])
    merged_ana = sorted(as_ana + r_ana, key=lambda ana_rel: ana_rel['utterance_id'])
    
    assert len(merged_trn) == len(merged_ana), 'merged trn and merged ana utterance sequences differ in length: {0} vs. {1}'.format(len(merged_trn), len(merged_ana))
    
    utterance_ids = [t_rel['utterance_id'] for t_rel in merged_trn]
    merged_rels = []
    for u_id in utterance_ids:
        matching_trns = [trn_rel for trn_rel in merged_trn if trn_rel['utterance_id'] == u_id]
        assert len(matching_trns) == 1, 'Utterance id {0} does not have exactly one matching trn relation, instead has {1}'.format(u_id, matching_trns)
        trn_rel = matching_trns[0]
        
        matching_anas = [ana_rel for ana_rel in merged_ana if ana_rel['utterance_id'] == u_id]
        assert len(matching_anas) == 1, 'Utterance id {0} does not have exactly one matching ana relation, instead has {1}'.format(u_id, matching_anas)
        ana_rel = matching_anas[0]
        
        merged_rel = dictionary_product(trn_rel, ana_rel)
        merged_rels.append(merged_rel)
    
    return merged_rels

In [133]:
get_transcriptions_bbn('05863')

[{'utterance_id': 'fe_03_05863-A-0001',
  'speech_raw': 'HI [LAUGHTER]',
  'speech': 'HI',
  'channel': 1,
  'start': 1360,
  'type': 'NIST_1A',
  'end': 12320,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 10960},
 {'utterance_id': 'fe_03_05863-A-0002',
  'speech_raw': 'HI MY NAME IS ZELDA',
  'speech': 'HI MY NAME IS ZELDA',
  'channel': 1,
  'start': 32560,
  'type': 'NIST_1A',
  'end': 46640,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 14080},
 {'utterance_id': 'fe_03_05863-A-0003',
  'speech_raw': 'HI MONIQUE',
  'speech': 'HI MONIQUE',
  'channel': 1,
  'start': 59760,
  'type': 'NIST_1A',
  'end': 69760,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 10000},
 {'utterance_id': 'fe_03_05863-A-0004',
  'speech_raw': "I'M FINE AND YOU",
  'speech': "I'M FINE AND YOU",
  'channel': 1,
  'start': 77280,
  'type': 'NIST_1A',
  'end': 88320,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 11040},
 {'utterance_id': 'fe_03_05863-A-0005',
  'speech_raw': 'GOOD',
  'speech': 'GOOD',
  'c

limit_output extension: Maximum message size of 20000 exceeded with 62534 characters

### Some summary / sanity-checking statistics

How many calls are in the bbn transcripts?

In [134]:
call_ids_bbn = [c['call_id'] for c in bbn_call_relation]
len(call_ids_bbn)
len(call_ids)
len(call_ids) - len(call_ids_bbn)

5120

5849

729

In [135]:
set(call_ids) - set(call_ids_bbn)

{'05851',
 '05852',
 '05853',
 '05854',
 '05855',
 '05856',
 '05857',
 '05858',
 '05859',
 '05860',
 '05861',
 '05862',
 '05864',
 '05865',
 '05866',
 '05867',
 '05868',
 '05869',
 '05870',
 '05871',
 '05872',
 '05873',
 '05874',
 '05875',
 '05877',
 '05878',
 '05879',
 '05880',
 '05881',
 '05882',
 '05883',
 '05884',
 '05885',
 '05886',
 '05890',
 '05891',
 '05892',
 '05893',
 '05894',
 '05895',
 '05896',
 '05897',
 '05899',
 '05900',
 '05901',
 '05902',
 '05903',
 '05904',
 '05905',
 '05906',
 '05907',
 '05908',
 '05909',
 '05910',
 '05911',
 '05912',
 '05913',
 '05914',
 '05915',
 '05916',
 '05917',
 '05918',
 '05919',
 '05920',
 '05921',
 '05922',
 '05923',
 '05924',
 '05925',
 '05926',
 '05927',
 '05928',
 '05929',
 '05930',
 '05931',
 '05932',
 '05933',
 '05934',
 '05935',
 '05936',
 '05937',
 '05938',
 '05939',
 '05940',
 '05941',
 '05942',
 '05943',
 '05944',
 '05945',
 '05946',
 '05947',
 '05948',
 '05949',
 '05950',
 '05951',
 '05952',
 '05953',
 '05954',
 '05955',
 '05956',


According to the documentation, only 300 conversations should be missing from the `bbn` transcriptions - those prefixed with `114`, `115`, and `116`. What's the story on the other 429????

How many word tokens are in the `bbn` transcripts?

In [136]:
def count_word_tokens_in_call_bbn(call_id):
    return sum([count_word_tokens_in_line(l) for l in get_transcriptions_bbn(call_id)])

#31.0s on wittgenstein
sum([count_word_tokens_in_call_bbn(call_id) for call_id in call_ids_bbn])

9676792

How many word types are in the `bbn` transcripts?

In [137]:
def word_types_in_call_bbn(call_id):
    return union([word_types_in_line(l) for l in get_transcriptions_bbn(call_id)])

#42.2s on wittgenstein
vocabulary_bbn = union([word_types_in_call_bbn(c) for c in call_ids_bbn])
len(vocabulary_bbn)

46160

In [138]:
list(vocabulary_bbn)[:100]

['',
 'BODYBUILDING',
 'SHOPAHOLIC',
 'EMBODIED',
 'BUTTING',
 'PURCHASER',
 'ECOMASS',
 'CHURNED',
 'DESK',
 'PIMPLE',
 'ABSURDEST',
 'JUDE',
 'TRANSOM',
 'JOURNALS',
 'LOBES',
 'UNT-',
 'SCANDALOUS',
 'URBANA-CHAMPAIGN',
 'PHONING',
 'VICARIOUS',
 'MONSON',
 'COUNTERBALANCES',
 'DICK',
 'SIZED',
 'QUEUES',
 'ESPOUSED',
 'R._I._C._O.',
 'PLAY-',
 'SABLE',
 "RILEY'S",
 'CATARACTS',
 'RULERS',
 'YAH-',
 'GORDON',
 'ESTA',
 'NICKELODEON',
 'SEWED',
 "DON'-",
 'PING',
 'POSSIBLY',
 'THETA',
 'OUTLYING',
 'CYNIC-',
 'FOODS',
 'TEMPTING',
 'APPALLED',
 'BEETS',
 'DRY',
 'STEPDAD',
 'BROADCASTING',
 'VERONIQUE',
 'LICKED',
 'INVACC-',
 "TRINITY'S",
 'UNLIMITED',
 'ATTRIBUTES',
 "MOM'S",
 'TRAITOR',
 "MARJIANA'S",
 'RUNNING',
 'DUDDY',
 'MESCULN',
 "MINNESOTA'S",
 'GINOS',
 'QUOTAS',
 'PAULINE',
 'PIGGY',
 'PLIANT',
 "WOULDN'T",
 'DEFORMED',
 'HAWTHORN',
 'FETTUCCINE',
 'ORGIES',
 'COPYCATS',
 'OCCUPATIONAL',
 'JETS',
 'FAIR-WEATHER',
 'SODOMY',
 'IMBIBE',
 'ISLANDERS',
 'HISSELF',
 'SCUZZY',

In [139]:
%%capture
#cell magic above suppresses useless output of file.write

vocabulary_bbn_fn = 'fisher_vocabulary_bbn.txt'

vocabulary_bbn_fp = os.path.join(repo_dir, vocabulary_bbn_fn)

#5.02s on wittgenstein
with open(vocabulary_bbn_fp, 'w') as file:
    for w in vocabulary_bbn:
        file.write("{0}\n".format(w));

In [140]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_vocabulary_bbn.txt | head -20

     1	
     2	BODYBUILDING
     3	SHOPAHOLIC
     4	EMBODIED
     5	BUTTING
     6	PURCHASER
     7	ECOMASS
     8	CHURNED
     9	DESK
    10	PIMPLE
    11	ABSURDEST
    12	JUDE
    13	TRANSOM
    14	JOURNALS
    15	LOBES
    16	UNT-
    17	SCANDALOUS
    18	URBANA-CHAMPAIGN
    19	PHONING
    20	VICARIOUS
cat: write error: Broken pipe


### Naive utterance sequence

In [141]:
def get_utterances_from_call_bbn(call_id):
    return [l['speech'] for l in get_transcriptions_bbn(call_id)]

In [142]:
call_ids_bbn[0]

'05863'

In [143]:
get_utterances_from_call_bbn('05863')[:10]

['HI',
 'HI MY NAME IS ZELDA',
 'HI MONIQUE',
 "I'M FINE AND YOU",
 'GOOD',
 'OKAY UM',
 'I WOULD NOT DO IT',
 "RIGHT I WOULDN'T UM DO YOU KID- ESP- UH TODAY I MEAN",
 "PEOPLE LOOK AT MONEY MONEY IS EVERYTHING IT'S LIKE THEY THINK THAT MONEY'S GOING TO RESOLVE ALL THEIR PROBLEMS AND A MILLION DOLLARS TODAY IS NOT A LOT OF MONEY AND I WOULD AND THAT'S A BETRAYAL TO ME",
 'RIGHT RIGHT']

In [144]:
#35.3s on wittgenstein
utterances_bbn = []
for c in call_ids_bbn:
    utterances_bbn.extend( get_utterances_from_call_bbn(c) )
len(utterances_bbn)

954017

In [145]:
utterances_bbn[0]

'HI'

In [146]:
%%capture
#cell magic above suppresses useless output of file.write

fisher_utterances_bbn_fn = 'fisher_utterances_bbn.txt'

fisher_utterances_bbn_path = os.path.join(repo_dir, fisher_utterances_bbn_fn)

#1m 46.3s on wittgenstein
with open(fisher_utterances_bbn_path, 'w') as file:
    for utterance in utterances_bbn:
        file.write("{0}\n".format(utterance));

In [147]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_bbn.txt | head -20

     1	HI
     2	HI MY NAME IS ZELDA
     3	HI MONIQUE
     4	I'M FINE AND YOU
     5	GOOD
     6	OKAY UM
     7	I WOULD NOT DO IT
     8	RIGHT I WOULDN'T UM DO YOU KID- ESP- UH TODAY I MEAN
     9	PEOPLE LOOK AT MONEY MONEY IS EVERYTHING IT'S LIKE THEY THINK THAT MONEY'S GOING TO RESOLVE ALL THEIR PROBLEMS AND A MILLION DOLLARS TODAY IS NOT A LOT OF MONEY AND I WOULD AND THAT'S A BETRAYAL TO ME
    10	RIGHT RIGHT
    11	NO
    12	I THINK HUMAN BEINGS ARE PR- UM FRIENDSHIP RELAT- RELATIONSHIPS WITH PEOPLE ARE A LOT MORE IMPORTANT THAN MONEY
    13	THEN YOU WEREN'T REALLY THEIR BEST FRIEND EITHER
    14	RIGHT RIGHT
    15	NO
    16	MM
    17	I DON'T THINK ANY MONEY IS WORTH ANYONE'S LIFE UH A HUMAN A PERSON'S LIFE IS WORTH EVERYTHING
    18	AMEN IS RIGHT
    19	I MEAN I- THERE'S NOT REALLY MUCH THAT WE CAN DEBATE ON THIS BECAUSE WE REALLY AGREED ON IT YOU KNOW
    20	RIGHT
cat: write error: Broken pipe


### All transcriptions

In [148]:
bbn_call_relation[:3]

[{'call_id': '05863',
  'trn_fn': 'fe_03_05863.trn',
  'ana_fn': 'fe_03_05863.ana',
  'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
  'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
  'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
  'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'},
 {'call_id': '05876',
  'trn_fn': 'fe_03_05876.trn',
  'ana_fn': 'fe_03_05876.ana',
  'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05876.trn',
  'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_

In [149]:
call_ids_bbn[:3]

['05863', '05876', '05887']

In [150]:
get_transcriptions_bbn('05863')[:3]

[{'utterance_id': 'fe_03_05863-A-0001',
  'speech_raw': 'HI [LAUGHTER]',
  'speech': 'HI',
  'channel': 1,
  'start': 1360,
  'type': 'NIST_1A',
  'end': 12320,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 10960},
 {'utterance_id': 'fe_03_05863-A-0002',
  'speech_raw': 'HI MY NAME IS ZELDA',
  'speech': 'HI MY NAME IS ZELDA',
  'channel': 1,
  'start': 32560,
  'type': 'NIST_1A',
  'end': 46640,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 14080},
 {'utterance_id': 'fe_03_05863-A-0003',
  'speech_raw': 'HI MONIQUE',
  'speech': 'HI MONIQUE',
  'channel': 1,
  'start': 59760,
  'type': 'NIST_1A',
  'end': 69760,
  'audio_fn': 'fe_03_05863.sph',
  'duration': 10000}]

In [151]:
bbn_call_relation[0]

{'call_id': '05863',
 'trn_fn': 'fe_03_05863.trn',
 'ana_fn': 'fe_03_05863.ana',
 'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
 'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
 'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
 'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'}

In [152]:
#39.0s on wittgenstein

bbn_transcription_relation = []

for call_relation in bbn_call_relation:
    for transcription_relation in get_transcriptions_bbn(call_relation['call_id']):
        new_transcription_rel = dictionary_product(transcription_relation, call_relation)
        bbn_transcription_relation.append(new_transcription_rel)

In [153]:
len(bbn_transcription_relation)

954017

In [154]:
bbn_transcription_relation[:5]

[{'utterance_id': 'fe_03_05863-A-0001',
  'channel': 1,
  'end': 12320,
  'start': 1360,
  'type': 'NIST_1A',
  'speech': 'HI',
  'audio_fn': 'fe_03_05863.sph',
  'duration': 10960,
  'speech_raw': 'HI [LAUGHTER]',
  'call_id': '05863',
  'auto-segmented_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana',
  'trn_fn': 'fe_03_05863.trn',
  'auto-segmented_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn',
  'ana_fn': 'fe_03_05863.ana',
  'rejected_trn_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn',
  'rejected_ana_location': '/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana'},
 {'utterance_id': 'fe_03_05863-A-0002',
  'channel': 1,
  'end': 46640,
  'start': 32560,
  'type': 'NIST_1A',
  'speech': 'HI MY NAME I

In [155]:
# import json

In [156]:
fisher_utterances_relation_bbn_fn = 'fisher_utterances_relation_bbn.json'

fisher_utterances_relation_bbn_path = os.path.join(repo_dir, fisher_utterances_relation_bbn_fn)

#32.5s on wittgenstein
with open(fisher_utterances_relation_bbn_path, 'w', encoding='utf8') as json_file:
    json.dump(bbn_transcription_relation, json_file, indent=1, ensure_ascii=False)

In [157]:
%cat -n /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_relation_bbn.json | head -20

     1	[
     2	 {
     3	  "utterance_id": "fe_03_05863-A-0001",
     4	  "channel": 1,
     5	  "end": 12320,
     6	  "start": 1360,
     7	  "type": "NIST_1A",
     8	  "speech": "HI",
     9	  "audio_fn": "fe_03_05863.sph",
    10	  "duration": 10960,
    11	  "speech_raw": "HI [LAUGHTER]",
    12	  "call_id": "05863",
    13	  "auto-segmented_ana_location": "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.ana",
    14	  "trn_fn": "fe_03_05863.trn",
    15	  "auto-segmented_trn_location": "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/auto-segmented/fe_03_05863.trn",
    16	  "ana_fn": "fe_03_05863.ana",
    17	  "rejected_trn_location": "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.trn",
    18	  "rejected_ana_location": "/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058/rejected/fe_03_05863.ana"
    19	

In [158]:
os.getcwd()

'/mnt/truffle/corpora/fisher_english_transcripts/fe_03_p2_tran/data/bbn_orig/058'

# Using `kenlm`

In [159]:
os.chdir(repo_dir)
os.getcwd()

'/mnt/cube/home/AD/emeinhar/fisher-lm'

In [160]:
%ls

'Creating an n-gram model from the Fisher Corpus.ipynb'
 fisher_utterances_bbn.txt
 fisher_utterances_main.txt
 fisher_utterances_relation_bbn.json
 fisher_utterances_relation_main.json
 fisher_vocabulary_bbn.txt
 fisher_vocabulary_main.txt


## Check for format complaints

Below is a dummy run, where we create a unigram model from 
 - fisher_utterances_main.txt
 - fisher_utterances_bbn.txt
to see if kenlm complains about the format of the files.

In [161]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 1 --text fisher_utterances_main.txt --arpa fisher_utterances_main_1gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 11032648 types 48799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:585588
Statistics:
1 48799 D1=0.580525 D2=0.987631 D3+=1.43469
Memory estimate for binary LM:
type      kB
probing 2096 assuming -p 1.5
probing 2287 assuming -r models -p 1.5
trie    1423 without quantization
trie    1285 assuming -q 8 -b 8 quantization 
trie    1423 assuming -a 22 array pointer compression
trie    1285 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:585588
=== 4/5 Calculating and writing order-interpolated probabilities ===
Chain sizes: 1:585588
=== 5/5 Writing ARPA 

In [162]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 1 --text fisher_utterances_bbn.txt --arpa fisher_utterances_bbn_1gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_bbn.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 9641143 types 46162
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:553944
Statistics:
1 46162 D1=0.574371 D2=0.999083 D3+=1.40785
Memory estimate for binary LM:
type      kB
probing 1983 assuming -p 1.5
probing 2163 assuming -r models -p 1.5
trie    1346 without quantization
trie    1216 assuming -q 8 -b 8 quantization 
trie    1346 assuming -a 22 array pointer compression
trie    1216 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:553944
=== 4/5 Calculating and writing order-interpolated probabilities ===
Chain sizes: 1:553944
=== 5/5 Writing ARPA mo

## Timing tests

Estimating a unigram model takes about 9s. Let's see how long progressively larger models take to estimate.

In [163]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 2 --text fisher_utterances_main.txt --arpa fisher_utterances_main_2gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 11032648 types 48799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:585588 2:108102098944
Statistics:
1 48799 D1=0.587025 D2=0.971389 D3+=1.46086
2 984993 D1=0.692134 D2=1.06902 D3+=1.38789
Memory estimate for binary LM:
type       kB
probing 18553 assuming -p 1.5
probing 18743 assuming -r models -p 1.5
trie     6794 without quantization
trie     4030 assuming -q 8 -b 8 quantization 
trie     6794 assuming -a 22 array pointer compression
trie     4030 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:585588 2:15759888
----5---10---15---20---25---30---35

In [164]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 3 --text fisher_utterances_main.txt --arpa fisher_utterances_main_3gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 11032648 types 48799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:585588 2:37600731136 3:70501367808
Statistics:
1 48799 D1=0.587025 D2=0.971389 D3+=1.46086
2 984993 D1=0.704158 D2=1.08091 D3+=1.40055
3 3730613 D1=0.78634 D2=1.09668 D3+=1.32213
Memory estimate for binary LM:
type    MB
probing 87 assuming -p 1.5
probing 93 assuming -r models -p 1.5
trie    33 without quantization
trie    18 assuming -q 8 -b 8 quantization 
trie    32 assuming -a 22 array pointer compression
trie    16 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:585588 2:15759888

In [165]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 4 --text fisher_utterances_main.txt --arpa fisher_utterances_main_4gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 11032648 types 48799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:585588 2:18400356352 3:34500669440 4:55201071104
Statistics:
1 48799 D1=0.587025 D2=0.971389 D3+=1.46086
2 984993 D1=0.704158 D2=1.08091 D3+=1.40055
3 3730613 D1=0.799743 D2=1.11614 D3+=1.36756
4 6586987 D1=0.869439 D2=1.16026 D3+=1.31928
Memory estimate for binary LM:
type     MB
probing 222 assuming -p 1.5
probing 249 assuming -r models -p 1.5
trie     95 without quantization
trie     50 assuming -q 8 -b 8 quantization 
trie     86 assuming -a 22 array pointer compression
trie     42 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and

In [166]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/lmplz -o 5 --text fisher_utterances_main.txt --arpa fisher_utterances_main_5gram.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /mnt/cube/home/AD/emeinhar/fisher-lm/fisher_utterances_main.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 11032648 types 48799
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:585588 2:10546545664 3:19774775296 4:31639638016 5:46141140992
Statistics:
1 48799 D1=0.587025 D2=0.971389 D3+=1.46086
2 984993 D1=0.704158 D2=1.08091 D3+=1.40055
3 3730613 D1=0.799743 D2=1.11614 D3+=1.36756
4 6586987 D1=0.882026 D2=1.19504 D3+=1.37969
5 7911211 D1=0.93335 D2=1.25853 D3+=1.36473
Memory estimate for binary LM:
type     MB
probing 395 assuming -p 1.5
probing 460 assuming -r models -p 1.5
trie    182 without quantization
trie     97 assuming -q 8 -b 8 quantization 
trie    162 assuming -a 22 array pointer compression
trie     77 assuming -a 22 -q 8 -b 8 array po

## Query an n-gram model

In [167]:
%ls *gram.arpa

fisher_utterances_bbn_1gram.arpa   fisher_utterances_main_3gram.arpa
fisher_utterances_main_1gram.arpa  fisher_utterances_main_4gram.arpa
fisher_utterances_main_2gram.arpa  fisher_utterances_main_5gram.arpa


In [168]:
!echo "how's it going" | /home/AD/emeinhar/GitHub/kenlm/build/bin/query fisher_utterances_main_5gram.arpa

Loading the LM will be faster if you build a binary file.
Reading fisher_utterances_main_5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
how's=3373 2 -3.8049774	it=82 3 -0.2931349	going=158 4 -0.057476994	</s>=2 5 -0.16472308	Total: -4.320312 OOV: 0
Perplexity including OOVs:	12.024803987034101
Perplexity excluding OOVs:	12.024803987034101
OOVs:	0
Tokens:	4
Name:query	VmPeak:435736 kB	VmRSS:5036 kB	RSSMax:410484 kB	user:7.15533	sys:0.071953	CPU:7.22731	real:7.22986


In [169]:
!/home/AD/emeinhar/GitHub/kenlm/build/bin/build_binary fisher_utterances_main_5gram.arpa fisher_utterances_main_5gram.mmap

Reading fisher_utterances_main_5gram.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [170]:
!echo "how's it going" | /home/AD/emeinhar/GitHub/kenlm/build/bin/query fisher_utterances_main_5gram.mmap

This binary file contains probing hash tables.
how's=3373 2 -3.8049774	it=82 3 -0.2931349	going=158 4 -0.057476994	</s>=2 5 -0.16472308	Total: -4.320312 OOV: 0
Perplexity including OOVs:	12.024803987034101
Perplexity excluding OOVs:	12.024803987034101
OOVs:	0
Tokens:	4
Name:query	VmPeak:433692 kB	VmRSS:4840 kB	RSSMax:410048 kB	user:0.003771	sys:0.026403	CPU:0.0302105	real:0.0284499
